In [1]:
import requests
import pandas as pd

import tqdm

import random
import time

from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv("data/apartamentos_incompleto.csv")
print(df.shape)
df.head()

(30748, 4)


,Unnamed: 0,nome,valor_venda,url
0,0,Tower duplex 90m2,550000,https://www.wimoveis.com.br/propriedades/tower...
1,1,SELECTION 36,2150000,https://www.wimoveis.com.br/propriedades/selec...
2,2,Elevador + Apartamento de 3 Quartos + Excelent...,420000,https://www.wimoveis.com.br/propriedades/eleva...
3,3,"01, 02 QUARTOS E LOJA COMERCIAL - MEET NOROESTE",332420,https://www.wimoveis.com.br/propriedades/01-02...
4,4,Elegance Park Sul de 03 Suítes,1333800,https://www.wimoveis.com.br/propriedades/elega...


In [3]:
df = df.drop(['Unnamed: 0'],axis=1)
df['condominio'] = ''
df['IPTU'] = ''
df['lat'] = ''
df['long'] = ''
df['area_total'] = ''
df['area_util'] = ''
df['banheiros'] = ''
df['vagas'] = ''
df['quartos'] = ''
df['suites'] = ''
df['idade'] = ''
df['endereco'] = ''
df['bairro'] = ''
df['idade_anuncio'] = ''

In [4]:
for i, row in tqdm.tqdm_notebook(df.iterrows(), total=df.shape[0]):
    try:
        
        tqdm.tqdm.write(str(i + 1) +" Buscando item " + row.url)
        response = requests.get(row.url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Condominio e IPTU
        valores = soup.findAll("div",{"class":"block-expensas block-row"})
        if(len(valores)>0):
            for item in valores:
                #tqdm.tqdm.write(str(item))
                if("Condomínio" in str(item) ):
                    #print(item.find("span").text)
                    df.at[i,'condominio'] = item.find("span").text
                else:
                    df.at[i,'IPTU'] = item.find("span").text
                    
        # latlong
        script = soup.findAll("script")[2].string
        lat = script[script.find("mapLat") + 9:script.find("mapLat") + 29].replace(",","")
        df.at[i,'lat'] = lat
        
        
        long = script[script.find("mapLng") + 9:script.find("mapLng") + 29].replace(",","")
        df.at[i,'long'] = long
        
        # icones
        icon_total = soup.find("i",{"class":"icon-f icon-f-stotal"})
        if icon_total is not None:
            area_total = icon_total.findNext('b').text
            df.at[i,'area_total'] = area_total
        
        icon_util = soup.find("i",{"class":"icon-f icon-f-scubierta"})
        if icon_util is not None:
            area_util = icon_util.findNext('b').text
            df.at[i,'area_util'] = area_util
            
        icon_banheiro = soup.find("i",{"class":"icon-f icon-f-bano"})
        if icon_banheiro is not None:
            qtd_banheiros = icon_banheiro.findNext('b').text
            df.at[i,'banheiros'] = qtd_banheiros
            
        icon_vagas = soup.find("i",{"class":"icon-f icon-f-cochera"})
        if icon_vagas is not None:
            qtd_vagas = icon_vagas.findNext('b').text
            df.at[i,'vagas'] = qtd_vagas
        
        icon_quartos = soup.find("i",{"class":"icon-f icon-f-dormitorio"})
        if icon_quartos is not None:
            qtd_quartos = icon_quartos.findNext('b').text
            df.at[i,'quartos'] = qtd_quartos
        
        icon_suites = soup.find("i",{"class":"icon-f icon-f-toilete"})
        if icon_suites is not None:
            qtd_suites = icon_suites.findNext('b').text
            df.at[i,'suites'] = qtd_suites
        
        icon_idade = soup.find("i",{"class":"icon-f icon-f-antiguedad"})
        if icon_idade is not None:
            idade = icon_idade.findNext('b').text
            df.at[i,'idade'] = idade
        
        h2_endereco = soup.find("h2",{"class":"title-location"})
        if h2_endereco is not None:
            df.at[i,'endereco'] = h2_endereco.text
            df.at[i,'bairro'] = h2_endereco.findNext("span").text
        
        h5_idade = soup.find("h5",{"class":"section-date css-float-r"})
        if h5_idade is not None:
            df.at[i,'idade_anuncio'] = h5_idade.text.replace("\n Publicado há ","").replace(" dias\n","")
        
    except Exception as e:
        print("exceção ao buscar dados. Mensagem:", str(e))
        pass
    
    #dorme
    aleatorio = random.randint(1, 5)
    tqdm.tqdm.write("agora vou aguardar: " + str(aleatorio))
    time.sleep(aleatorio)

print(df.shape)
df.head(10)

1 Buscando item https://www.wimoveis.com.br/propriedades/tower-duplex-90m2-2933747524.html
agora vou aguardar: 3
2 Buscando item https://www.wimoveis.com.br/propriedades/selection-36-2942138179.html
agora vou aguardar: 4
3 Buscando item https://www.wimoveis.com.br/propriedades/elevador-apartamento-de-3-quartos-excelente-2942214874.html
agora vou aguardar: 4
4 Buscando item https://www.wimoveis.com.br/propriedades/01-02-quartos-e-loja-comercial-meet-noroeste-2938988272.html
agora vou aguardar: 5
5 Buscando item https://www.wimoveis.com.br/propriedades/elegance-park-sul-de-03-suites-2942383819.html
agora vou aguardar: 1
6 Buscando item https://www.wimoveis.com.br/propriedades/qi-31-jardins-life-reformadissimo-localizacao-e-tudo-!-2938838063.html
agora vou aguardar: 5
7 Buscando item https://www.wimoveis.com.br/propriedades/apartamento-1-qto-a-venda-no-hotel-bonaparte-2941485429.html
agora vou aguardar: 3
(30748, 17)


,nome,valor_venda,url,condominio,IPTU,lat,long,area_total,area_util,banheiros,vagas,quartos,suites,idade,endereco,bairro,idade_anuncio
0,Tower duplex 90m2,550000,https://www.wimoveis.com.br/propriedades/tower...,R$ 730,R$ 690,-15.842635250484830,-48.024645807548495,89m²,89m²,3,2,2,2,6,"Qd 206, Aguas Sul, Águas Claras",", Aguas Sul, Águas Claras",51
1,SELECTION 36,2150000,https://www.wimoveis.com.br/propriedades/selec...,,,-15.742129999999999,-47.910919999999997,224m²,224m²,6,3,4,4,4,"Quadra SQNW 110, Noroeste, Brasília",", Noroeste, Brasília",5
2,Elevador + Apartamento de 3 Quartos + Excelent...,420000,https://www.wimoveis.com.br/propriedades/eleva...,,,-15.739203900000000,-47.902473900000020,,65m²,2,0,3,0,,SCRN 706/707 BLOCO F - COMPLETO ARMÁRIOS- EXCE...,", Asa Norte, Brasília",8
3,"01, 02 QUARTOS E LOJA COMERCIAL - MEET NOROESTE",332420,https://www.wimoveis.com.br/propriedades/01-02...,,,-15.748856500000000,-47.896520000000010,,,,,,,,,,13
4,Elegance Park Sul de 03 Suítes,1333800,https://www.wimoveis.com.br/propriedades/elega...,,,-15.827143900000000,-47.952883600000010,117m²,117m²,5,2,3,3,1,"Sgcv 23, Park Sul, Brasília",", Park Sul, Brasília",7
5,QI 31 Jardins Life Reformadíssimo Localização ...,580000,https://www.wimoveis.com.br/propriedades/qi-31...,,,-15.840017800000000,-47.975002600000038,,68m²,2,1,2,1,3,"QUADRA QI 31 LOTES 2/4, Guara II, Guará",", Guara II, Guará",5
6,APARTAMENTO 1 QTO À VENDA NO HOTEL BONAPARTE,255000,https://www.wimoveis.com.br/propriedades/apart...,R$ 1.585,R$ 159,-15.792914000000000,-47.892922999999996,95m²,54m²,1,1,1,1,1,"SHS Quadra 2 Bloco J, Asa Sul, Brasília",", Asa Sul, Brasília",118
7,4 Suítes - Noroeste - Parque das Tulipas,1872821,https://www.wimoveis.com.br/propriedades/4-sui...,,,,,,,,,,,,,,
8,Aguas Claras 197 metros Cozinha em ilha: para...,1010000,https://www.wimoveis.com.br/propriedades/aguas...,,,,,,,,,,,,,,
9,VISION E FUSION: 10ª ANDAR COM LINDA VISTA,290000,https://www.wimoveis.com.br/propriedades/visio...,,,,,,,,,,,,,,
